In [3]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob

In [6]:
# get data using a for loop and append to dfs
path =r'C:\Users\BenG\Documents\GitHub\Battle_Of_The_Alamo'
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# Concatenate all data into one DataFrame
auth4_data = pd.concat(dfs, ignore_index=True)


In [44]:
#Initial Inspection of Dataframe also used auth4_data to see row count
auth4_data() #.head()

,time,src_comp,dst_comp,auth_type,logon_type,auth_orientation,succ_fail,dst_user
0,726131,C19193,C625,Kerberos,Network,LogOn,Success,U9040
1,726131,C19290,C529,Kerberos,Network,LogOn,Success,U9093
2,726131,C19290,C586,Kerberos,Network,LogOn,Success,U9093
3,726131,C19359,C5618,NTLM,Network,LogOn,Success,U9136
4,726131,C19632,C2162,Kerberos,Network,LogOn,Success,U9292


In [10]:
#Convert ? to standard NaN for further processing
auth4_clean= auth4_data.replace('?', np.NaN)
auth4_clean.head()

,time,src_user@dom,dst_user@dom,src_comp,dst_comp,auth_type,logon_type,auth_orientation,succ_fail
0,726131,U9040@DOM1,U9040@DOM1,C19193,C625,NaN,NaN,TGS,Success
1,726131,U9040@DOM1,U9040@DOM1,C19193,C625,Kerberos,Network,LogOn,Success
2,726131,U9040@DOM1,U9040@DOM1,C529,C529,NaN,Network,LogOff,Success
3,726131,U9040@DOM1,U9040@DOM1,C625,C625,NaN,Network,LogOff,Success
4,726131,U9093@DOM1,U9093@DOM1,C19290,C529,Kerberos,Network,LogOn,Success


In [18]:
#Remove rows with NaN and reset index for new dataframe
auth4_drop = auth4_clean.dropna()
auth4_index = auth4_drop.reset_index(drop=True)
auth4_index.head()

,time,src_user@dom,dst_user@dom,src_comp,dst_comp,auth_type,logon_type,auth_orientation,succ_fail
0,726131,U9040@DOM1,U9040@DOM1,C19193,C625,Kerberos,Network,LogOn,Success
1,726131,U9093@DOM1,U9093@DOM1,C19290,C529,Kerberos,Network,LogOn,Success
2,726131,U9093@DOM1,U9093@DOM1,C19290,C586,Kerberos,Network,LogOn,Success
3,726131,U9136@DOM1,U9136@DOM1,C19359,C5618,NTLM,Network,LogOn,Success
4,726131,U9292@DOM1,U9292@DOM1,C19632,C2162,Kerberos,Network,LogOn,Success


In [20]:
#Split apart src_user + dst_user in train data for relevant info
auth4_index[['src_user', 'dom']] = auth4_index['src_user@dom'].str.split('@', expand=True)
auth4_index[['dst_user', 'dom2']] = auth4_index['dst_user@dom'].str.split('@', expand=True)

In [27]:
#Checking out multiple columns for how many values would encode to binary
#Also figured out that the number of distinct values between columns is around 3%, dst_user column preserves the distinct values
#auth4_index['src_comp'].value_counts()
#auth4_index['dst_comp'].value_counts()
match = auth4_index.apply(lambda x: x['src_user'] != x['dst_user'], axis=1)
match.value_counts()

False    4397616
True      127174
dtype: int64

In [29]:
#Inspect the src_comp and dst_comp columns for duplicates, True means there are 87% distinct values across the columns
match2 = auth4_index.apply(lambda x: x['src_comp'] != x['dst_comp'], axis=1)
match2.value_counts()

True     4002908
False     521882
dtype: int64

In [45]:
#Inspect train data and drop unnecessary and duplicated info
#auth4_index.head() has 4,524,790 rows after clean up, and the red team data (src_comp = C17693) is preserved within
auth4_data = auth4_index.drop(columns=['src_user@dom', 'dst_user@dom', 'src_user', 'dom', 'dom2'])
auth4_train = auth4_data[['time', 'dst_user', 'src_comp', 'dst_comp', 'auth_type', 'logon_type', 'auth_orientation', 'succ_fail']]
auth4_train.head()


,time,dst_user,src_comp,dst_comp,auth_type,logon_type,auth_orientation,succ_fail
0,726131,U9040,C19193,C625,Kerberos,Network,LogOn,Success
1,726131,U9093,C19290,C529,Kerberos,Network,LogOn,Success
2,726131,U9093,C19290,C586,Kerberos,Network,LogOn,Success
3,726131,U9136,C19359,C5618,NTLM,Network,LogOn,Success
4,726131,U9292,C19632,C2162,Kerberos,Network,LogOn,Success


In [48]:
#Spit out our cleaned up dataset to a CSV that can be read in by pandas
auth4_train.to_csv('auth4_training.csv')